In [2]:
cd ..

/Users/vijay/vijaygkd/deep-ttt


In [438]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as k
import mlflow

In [6]:
#%load_ext autoreload
%autoreload 2

In [1089]:
from dqn.environment import *
from dqn.agent import RandomAgent, QAgent
from dqn.model import DQN
from dqn.train import Memory, QLearning, play_games_against_random


In [1224]:
t = QLearning()

hey11


In [1225]:
t.net.model.summary()

Model: "model_63"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_states (InputLayer)      [(None, 9)]          0           []                               
                                                                                                  
 hidden_1 (Dense)               (None, 128)          1280        ['input_states[0][0]']           
                                                                                                  
 hidden_2 (Dense)               (None, 64)           8256        ['hidden_1[0][0]']               
                                                                                                  
 input_actions (InputLayer)     [(None, 1)]          0           []                               
                                                                                           

In [1226]:
with mlflow.start_run(run_name='test run 11 - q value output layer', description='') as run:
    t.train(10000)

100%|█████████████████████████████████████████████████████████████████| 10000/10000 [1:01:44<00:00,  2.70it/s]


## Gradient Calculations

In [1414]:
X = np.array([[1., 0., 2.],
             [0., 1., 0.],
             [0., 0., 3.]])
Xt = tf.convert_to_tensor(X)

W = np.array([[1., 0.],
             [1., 1.],
             [0., 1.]])
Wt = tf.convert_to_tensor(W)

In [1415]:
Y_hat = np.dot(X, W)
Y_hat

array([[1., 2.],
       [1., 1.],
       [0., 3.]])

In [1417]:
Y = np.array([[1., 3.],
              [1., 3.],
              [0., 0.]])

In [1419]:
(1+4+9)/6

2.3333333333333335

In [1418]:
mse = tf.keras.losses.MeanSquaredError()
mse(Y, Y_hat).numpy()

2.3333333333333335

In [1420]:
Y-Y_hat

array([[ 0.,  1.],
       [ 0.,  2.],
       [ 0., -3.]])

In [1424]:
np.dot(X.T, Y-Y_hat) * -2/6

array([[-0.        , -0.33333333],
       [-0.        , -0.66666667],
       [-0.        ,  2.33333333]])

In [1422]:
with tf.GradientTape(persistent=True) as g:
    g.watch([Xt, Wt])
    Yh = Xt @ Wt
    L = mse(Y, Yh)

In [1423]:
g.gradient(L, [Wt, Xt])

[<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
 array([[ 0.        , -0.33333333],
        [ 0.        , -0.66666667],
        [ 0.        ,  2.33333333]])>,
 <tf.Tensor: shape=(3, 3), dtype=float64, numpy=
 array([[ 0.        , -0.33333333, -0.33333333],
        [ 0.        , -0.66666667, -0.66666667],
        [ 0.        ,  1.        ,  1.        ]])>]

In [1285]:
L

<tf.Tensor: shape=(), dtype=float64, numpy=1.3333333333333333>

In [1299]:
np.dot(X.T, (Y-Y_hat)) * -2 / 6

array([[-0.33333333, -0.        ],
       [ 0.33333333, -0.33333333],
       [-1.66666667,  2.        ]])

In [1291]:
np.dot(Y_hat-Y, W.T) * 2 / 6

array([[-0.33333333, -0.33333333,  0.        ],
       [ 0.33333333,  0.        , -0.33333333],
       [-0.33333333,  0.33333333,  0.66666667]])

In [1303]:
np.dot(X, (Y-Y_hat)) * -2 / 6  # ---> Wrong

array([[-1.        ,  1.33333333],
       [ 0.33333333, -0.33333333],
       [-1.        ,  2.        ]])

In [1426]:

mse = tf.keras.losses.MeanSquaredError()


## Play against AI

In [1406]:
g = TicTacToe()

In [1412]:
g.execute_action(6)


(-1, 0)

In [1413]:
m, q = t.agent.play(g.get_current_state())
print(m, q)
g.execute_action(m)
g.print_board()

1 0.25119048
X X X 

- O X 

O - O 



In [1348]:
g.print_board()

- - - 

- - - 

- - - 



In [1161]:
s = play_games_against_random(t.agent, 100)

In [1162]:
s

{'win': 0.91, 'lost': 0.06, 'draw': 0.03}

In [1169]:
board_samples

[array([[ 1., -1.,  1., -1.,  0., -1.,  1.,  1., -1.]]),
 array([4.]),
 array([0.5]),
 array([[-1., -1., -1., -1., -1., -1., -1., -1., -1.]]),
 array([1.])]

### Test gradient updates